In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2
import random
from collections import Counter
import time
from datetime import datetime
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [ ]:
import os

clear_sky_images = os.listdir("resources/cloud-images/cloud-classifier-1/clear_sky")
cloud_images = os.listdir("resources/cloud-images/cloud-classifier-1/cloud")

labeled_data = {}

# Fine to label the images like this for now because there are only 2 classes, but best to automate this step using folder names next

for i in clear_sky_images:
    labeled_data[i] =  [{'label': 'clear_sky'}]

for i in cloud_images:
    labeled_data[i] =  [{'label': 'cloud'}]

print(len(labeled_data))

In [ ]:
train_images = []
train_labels = []
class_object = ['clear_sky', 'cloud']

start_datetime = datetime.now()

for (i, image_file) in enumerate(labeled_data):
    #read image
    path = 'resources/cloud-images/cloud-classifier-1/'+ labeled_data[image_file][0]['label'] + '/' + image_file
    print(path)
    image = cv2.imread(path)
    #make images gray
    plt.imshow(image)
    plt.show()
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #label image using the annotations
    label = class_object.index(labeled_data[image_file][0]['label'])
    tmp_label = labeled_data[image_file][0]['label']
    # resize image
    image = cv2.resize(image, (32, 32))
    # flatten the image
    pixels = image
    #Append flattened image to
    train_images.append(pixels)
    train_labels.append(label)
    print('Loaded...', '\U0001F483', 'Image', str(i+1), 'is a', tmp_label)
    plt.imshow(image)
    plt.show()

end_datetime = datetime.now()

In [ ]:
print('Image Processing Duration: ' + str(end_datetime-start_datetime))

train_images = np.array(train_images)
train_labels = np.array(train_labels)

train_images = train_images.reshape((train_images.shape[0], -1))


test_size = 0.3

train_samples, test_samples, train_samples_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=test_size, random_state=None)

In [ ]:
scaler = StandardScaler()

print(train_images)
X_train = scaler.fit_transform(train_images)
X_test = scaler.transform(test_samples)

y_train, y_test = train_labels, test_labels

clf = LogisticRegression(C=50.0 / len(X_train), penalty="l1", solver="saga", tol=0.1)
clf.fit(X_train, y_train)
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

In [ ]:
coef = clf.coef_.copy()
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(2):
    l1_plot = plt.subplot(2, 5, i + 1)
    l1_plot.imshow(
        coef[i].reshape(32, 32),
        interpolation="nearest",
        cmap=plt.cm.RdBu,
        vmin=-scale,
        vmax=scale,
    )
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel(f"Class {i}")
plt.suptitle("Classification vector for...")

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(coef.shape[0]):
    l1_plot = plt.subplot(1, coef.shape[0], i + 1)
    l1_plot.imshow(
        coef[i].reshape(32, 32),
        interpolation="nearest",
        cmap=plt.cm.RdBu,
        vmin=-scale,
        vmax=scale,
    )
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel(f"Class {i}")
plt.suptitle("Classification vector for...")

print(coef.shape)

plt.show()